In [26]:
# Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import os
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests

In [27]:
# Configure Chrome Webdriver

chrome_install = ChromeDriverManager().install()

In [28]:
# Initialize Chrome WebDriver
browser = webdriver.Chrome()


In [29]:
# Setup search parameters
city = "montreal"
product = "surron"
min_price = 100
max_price = 10000
days_listed = 60

In [ ]:
# Setup base URL
url = f'https://www.facebook.com/marketplace/{city}/search?query={product}&minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_listed}&exact=false'

# Visit website
browser.get(url)

# Close login pop-up
try:
    close_button = browser.find_element(By.XPATH, "//div[@aria-label='Close']")
    close_button.click()
    time.sleep(2)
except:
    pass

